## 不同分布下的结果

In [1]:
import os
import pandas as pd

# --- 1. 数据准备 ---
cur_path = os.getcwd()
# 假设路径结构，根据你的实际情况调整
data_dir = cur_path + '/../data/origin/' 
diff_noise_path = data_dir + 'new_attack.csv'


df = pd.read_csv(diff_noise_path)

# 添加 Safe 列 (Vectorized operations)
df['cmi_safe'] = df['cmi_max_iou'] < 0.5
df['fa_safe'] = df['fa_min_iou'] > 0.5

df = df[df['epsilon'] == '1/255']
new_data = []

# noise_type = {'normal',  'poisson', 'salt_and_pepper', 'uniform'}
# df = df[df['noise_type'].isin(noise_type)]
# --- 2. 循环计算 ---
for (model, epsilon, noise_type), group in df.groupby(['model', 'epsilon', 'noise_type']):
    print(f'Processing -> Model: {model}, Noise: {noise_type}')
    
    # CMI 计数器
    c_tp, c_tn, c_fp, c_fn = 0, 0, 0, 0
    # FA 计数器
    f_tp, f_tn, f_fp, f_fn = 0, 0, 0, 0
    
    # 按图片聚合 (要求所有 gt_index 都满足才算 safe)
    for image_id, sub_group in group.groupby('image'):
        
        odpv_df = sub_group[sub_group['method'] == 'odpv']
        rcp_df = sub_group[sub_group['method'] == 'rcp']
        
        if odpv_df.empty or rcp_df.empty:
            continue

        # === CMI 计算 ===
        # 预测值 (Prediction): ODPV
        pred_safe_cmi = odpv_df['cmi_safe'].all()
        # 真值 (Ground Truth): RCP
        gt_safe_cmi = rcp_df['cmi_safe'].all()

        if pred_safe_cmi and gt_safe_cmi:
            c_tp += 1  # Both Safe
        elif not pred_safe_cmi and not gt_safe_cmi:
            c_tn += 1  # Both Unsafe
        elif pred_safe_cmi and not gt_safe_cmi:
            c_fp += 1  # Pred Safe, GT Unsafe (误报)
        elif not pred_safe_cmi and gt_safe_cmi:
            c_fn += 1  # Pred Unsafe, GT Safe (漏报)

        # === FA 计算 ===
        pred_safe_fa = odpv_df['fa_safe'].all()
        gt_safe_fa = rcp_df['fa_safe'].all()

        if pred_safe_fa and gt_safe_fa:
            f_tp += 1
        elif not pred_safe_fa and not gt_safe_fa:
            f_tn += 1
        elif pred_safe_fa and not gt_safe_fa:
            f_fp += 1
        elif not pred_safe_fa and gt_safe_fa:
            f_fn += 1

    # --- 3. 指标计算函数 ---
    def calculate_metrics(tp, tn, fp, fn):
        total_images = tp + tn + fp + fn
        
        # 1. ODPV Safe Ratio (新增指标): ODPV认为是safe的总数 / 总图片数
        odpv_total_safe = tp + fp
        odpv_safe_ratio = odpv_total_safe / total_images if total_images > 0 else 0

        # 2. CAR (Precision): 二者都认为safe / rcp认为safe
        car = tp / tp + fp if odpv_total_safe > 0 else 0
        
        # 3. TPR (Recall): 分母是 RCP(GT) 为 safe
        gt_total_safe = tp + fn
        tpr = tp / gt_total_safe if gt_total_safe > 0 else 0
        
        # 4. TNR (Specificity): 分母是 RCP(GT) 为 unsafe
        gt_total_unsafe = tn + fp
        tnr = tn / gt_total_unsafe if gt_total_unsafe > 0 else '-'
        
        # 5. FPR: 1 - TNR
        fpr = fp / gt_total_unsafe if gt_total_unsafe > 0 else 0
        
        # 6. FNR: 1 - TPR
        fnr = fn / gt_total_safe if gt_total_safe > 0 else 0
        
        return odpv_safe_ratio, car, tpr, tnr, fpr, fnr

    # 计算 CMI 指标
    c_ratio, c_car, c_tpr, c_tnr, c_fpr, c_fnr = calculate_metrics(c_tp, c_tn, c_fp, c_fn)
    # 计算 FA 指标
    f_ratio, f_car, f_tpr, f_tnr, f_fpr, f_fnr = calculate_metrics(f_tp, f_tn, f_fp, f_fn)

    new_data.append({
        'Model': model,
        # r'$\varepsilon$': epsilon,
        'Noise type': noise_type,
        
        # # --- CMI Metrics ---
        # 'cmi_odpv_ratio': c_ratio,  # 新增: ODPV认证率
        r'$\mathrm{CAR}_{\text{CMI}}$': f'{c_car * 100:.2f}%',
        r'$\mathrm{TPR}_{\text{CMI}}$': f'{c_tpr * 100:.2f}%',
        r'$\mathrm{TNR}_{\text{CMI}}$': f'{c_tnr * 100:.2f}%',
        r'$\mathrm{FPR}_{\text{CMI}}$': f'{c_fpr * 100:.2f}%',
        r'$\mathrm{FNR}_{\text{CMI}}$': f'{c_fnr * 100:.2f}%',
        
        # --- FA Metrics ---
        # 'fa_odpv_ratio': f_ratio,   # 新增: ODPV认证率
        r'$\mathrm{CAR}_{\text{FA}}$': f'{f_car * 100:.2f}%',
        r'$\mathrm{TPR}_{\text{FA}}$': f'{f_tpr * 100:.2f}%',
        r'$\mathrm{TNR}_{\text{FA}}$': f'{f_tnr * 100:.2f}%',
        r'$\mathrm{FPR}_{\text{FA}}$': f'{f_fpr * 100:.2f}%',
        r'$\mathrm{FNR}_{\text{FA}}$': f'{f_fnr * 100:.2f}%'
    })

new_df = pd.DataFrame(new_data)

# 格式化打印，保留4位小数方便查看
print(new_df.round(2))
print(new_df.to_markdown(index=False))
# 保存结果
# new_df.to_csv('metrics_with_ratio.csv', index=False)

Processing -> Model: yolo11x, Noise: laplace
Processing -> Model: yolo11x, Noise: normal
Processing -> Model: yolo11x, Noise: poisson
Processing -> Model: yolo11x, Noise: poisson_gaussian
Processing -> Model: yolo11x, Noise: salt_and_pepper
Processing -> Model: yolo11x, Noise: uniform
     Model        Noise type $\mathrm{CAR}_{\text{CMI}}$  \
0  yolo11x           laplace                     100.00%   
1  yolo11x            normal                     100.00%   
2  yolo11x           poisson                     100.00%   
3  yolo11x  poisson_gaussian                     100.00%   
4  yolo11x   salt_and_pepper                     100.00%   
5  yolo11x           uniform                     100.00%   

  $\mathrm{TPR}_{\text{CMI}}$ $\mathrm{TNR}_{\text{CMI}}$  \
0                      12.82%                     100.00%   
1                      45.00%                     100.00%   
2                      10.26%                     100.00%   
3                      10.26%                    